In [3]:
import numpy as np
import xarray as xr
import plotly.graph_objects as go
import uwacan

In [20]:
class ClassLimitCurve:
    def __init__(self, breakpoints, limits, default_frequencies=None):
        self.breakpoints = breakpoints
        self.limits = limits
        if default_frequencies is not None:
            self.data = xr.DataArray(self(default_frequencies), coords={"frequency": default_frequencies})

    def __array__(self):
        return self.data.__array__()

    @property
    def frequency(self):
        return self.data.frequency

    def __call__(self, frequency):
        conditions = [frequency < b for b in self.breakpoints] + [np.full(frequency.shape, True)]
        limits = [limit(frequency) for limit  in self.limits]
        levels = np.select(conditions, limits)
        return levels


bureau_veritas_advanced = ClassLimitCurve(
    breakpoints=[50, 1e3],
    limits=[
        lambda f: 169 - 2 * np.log10(f),
        lambda f: 165.6 - 20 * np.log10(f / 50),
        lambda f: 139.6 - 20 * np.log10(f / 1000),
    ],
    default_frequencies=np.geomspace(10, 50e3, 5000),
)

In [15]:
class ClassLimitCurve(uwacan.analysis.FrequencyData):
    def __init__(self, data=None, breakpoints=None, limits=None, default_frequencies=None):
        super().__init__(data)
        self.breakpoints = breakpoints
        self.limits = limits
        if default_frequencies is not None:
            self._data = xr.DataArray(self(default_frequencies), coords={"frequency": default_frequencies})

    # def __array__(self):
        # return self.data.__array__()

    # @property
    # def frequency(self):
        # return self.data.frequency

    def __call__(self, frequency):
        conditions = [frequency < b for b in self.breakpoints] + [np.full(frequency.shape, True)]
        limits = [limit(frequency) for limit  in self.limits]
        levels = np.select(conditions, limits)
        return levels


bureau_veritas_advanced = ClassLimitCurve(
    breakpoints=[50, 1e3],
    limits=[
        lambda f: 169 - 2 * np.log10(f),
        lambda f: 165.6 - 20 * np.log10(f / 50),
        lambda f: 139.6 - 20 * np.log10(f / 1000),
    ],
    default_frequencies=np.geomspace(10, 50e3, 5000),
)

In [57]:
def class_limit_curve(frequency, breakpoints, limits):
    conditions = [frequency < b for b in breakpoints] + [np.full(np.shape(frequency), True)]
    limits = [limit(frequency) if callable(limit) else limit for limit  in limits]
    levels = np.select(conditions, limits)
    try:
        wrapper = frequency.__array_wrap__
    except AttributeError:
        return levels
    else:
        return wrapper(levels)


def bureau_veritas_advanced(frequency=None):
    if frequency is None:
        frequency = 10 ** (np.arange(10, 48) / 10)  # Decidecade bands from 10 Hz to 50 kHz
        frequency = xr.DataArray(frequency, coords={"frequency": frequency})
    return class_limit_curve(
        frequency=frequency,
        breakpoints=[50, 1e3],
        limits=[
            169 - 2 * np.log10(frequency),
            # lambda f: 169 - 2 * np.log10(f),
            lambda f: 165.6 - 20 * np.log10(f / 50),
            lambda f: 139.6 - 20 * np.log10(f / 1000),
        ]
    )

In [60]:
    class_limit_curve(
        np.array([50, 100, 200, 400, 800, 1600]),
        [150, 800],
        [10, 20, 30],
    )

array([10, 10, 20, 20, 30, 30])

In [56]:
bureau_veritas_advanced()

<xarray.DataArray (frequency: 38)>
array([167.        , 166.8       , 166.6       , 166.4       ,
       166.2       , 166.        , 165.8       , 165.57940009,
       163.57940009, 161.57940009, 159.57940009, 157.57940009,
       155.57940009, 153.57940009, 151.57940009, 149.57940009,
       147.57940009, 145.57940009, 143.57940009, 141.57940009,
       139.6       , 137.6       , 135.6       , 133.6       ,
       131.6       , 129.6       , 127.6       , 125.6       ,
       123.6       , 121.6       , 119.6       , 117.6       ,
       115.6       , 113.6       , 111.6       , 109.6       ,
       107.6       , 105.6       ])
Coordinates:
  * frequency  (frequency) float64 10.0 12.59 15.85 ... 3.981e+04 5.012e+04

In [33]:
f = np.geomspace(1, 100_000, 1000)
go.Figure(
    [
        # go.Scatter(x=f, y=bureau_veritas_advanced(f)),
        go.Scatter(x=bureau_veritas_advanced().frequency, y=bureau_veritas_advanced()),
    ]
).update_xaxes(type="log")

In [31]:
breakpoints = [50, 1e3]
frequency = 10 ** (np.arange(10, 48) / 10)
frequency = xr.DataArray(frequency, coords={"frequency": frequency})
conditions = [frequency < b for b in breakpoints] + [np.full(frequency.shape, True)]
# np.argmax(np.stack(conditions), 0)
limits=[
    lambda f: 169 - 2 * np.log10(f),
    lambda f: 165.6 - 20 * np.log10(f / 50),
    lambda f: 139.6 - 20 * np.log10(f / 1000),
]
limits = [limit(frequency) for limit in limits]
levels = np.select(conditions, limits)
frequency.__array_wrap__(levels)
# xr_conds = xr.DataArray(conditions, dims=["conditions", "frequency"])
# conditions

<xarray.DataArray (frequency: 38)>
array([167.        , 166.8       , 166.6       , 166.4       ,
       166.2       , 166.        , 165.8       , 165.57940009,
       163.57940009, 161.57940009, 159.57940009, 157.57940009,
       155.57940009, 153.57940009, 151.57940009, 149.57940009,
       147.57940009, 145.57940009, 143.57940009, 141.57940009,
       139.6       , 137.6       , 135.6       , 133.6       ,
       131.6       , 129.6       , 127.6       , 125.6       ,
       123.6       , 121.6       , 119.6       , 117.6       ,
       115.6       , 113.6       , 111.6       , 109.6       ,
       107.6       , 105.6       ])
Coordinates:
  * frequency  (frequency) float64 10.0 12.59 15.85 ... 3.981e+04 5.012e+04